In [1]:
import json
import os
import time

def finder(json_file):
    # Need to change this line to be a variable passed to the function
    # I.E. Invoke the search based on feeding it a JSON
    f = open(json_file)

    # returns JSON object as
    # a dictionary
    data = json.load(f)

    print (data)

    # Get the folder to scan
    directory = (data['watch_folder'])

    print ('Will now search the directory ' + directory + ' and provide the relevant config flags:')

    # traverse whole directory
    for root, dirs, files in os.walk(directory):
        # select file name
        for file in files:
            # check the extension of files
            if file.endswith('.mkv') or file.endswith('.mp4'):
                # append the desired fields to the original json
                finder_data = {'file_path':root, 'file_name':file}
                finder_data.update(data)      
                print(json.dumps(finder_data, indent=3, sort_keys=True))
                yield(finder_data)

In [2]:
json_file = f'C:\\Users\\Chase\\media\\Newfolder\\AV1.json'
mygenerator = finder(json_file) 
for finder_data in mygenerator:
    print(finder_data)


{'config_name': 'default', 'watch_folder': 'C:\\Users\\Chase\\media\\Newfolder', 'ffmpeg_container': 'MKV', 'ffmpeg_container_string': 'matroska,webm', 'ffmpeg_video_codec': 'AV1', 'ffmpeg_video_string': '-c:v libsvtav1 -crf 20 -preset 4 -g 240 -pix_fmt yuv420p10le', 'ffmpeg_audio_codec': 'OPUS', 'ffmpeg_audio_string': '-c:a libopus', 'ffmpeg_subtitle_format': 'SRT', 'ffmpeg_subtitle_string': '-c:s srt'}
Will now search the directory C:\Users\Chase\media\Newfolder and provide the relevant config flags:
{
   "config_name": "default",
   "ffmpeg_audio_codec": "OPUS",
   "ffmpeg_audio_string": "-c:a libopus",
   "ffmpeg_container": "MKV",
   "ffmpeg_container_string": "matroska,webm",
   "ffmpeg_subtitle_format": "SRT",
   "ffmpeg_subtitle_string": "-c:s srt",
   "ffmpeg_video_codec": "AV1",
   "ffmpeg_video_string": "-c:v libsvtav1 -crf 20 -preset 4 -g 240 -pix_fmt yuv420p10le",
   "file_name": "A Certain Scientific Railgun - S00E01 - MMR - Much More Railgun.mkv",
   "file_path": "C:\\Us

In [13]:
import  json, subprocess, os
def ffprober(filename):
    
    # Uncomment to see the incoming JSON
    #print(json.dumps(filename, indent=3, sort_keys=True))
    
    # Need to assemble the full filepath to pass to FFprobe
    file_path = (filename["file_path"])
    file_name = (filename["file_name"])
    ffprobe_path = os.path.join(file_path,file_name)
    
    print ('Going to execute FFprobe on: ' + ffprobe_path)
    
    # Using subprocess to call FFprobe, get JSON back    
    cmnd = [f'ffprobe', '-loglevel', 'quiet', '-show_entries', 'format:stream=index,stream,codec_type,codec_name', '-of', 'json', ffprobe_path]
    p = subprocess.run(cmnd, capture_output=True).stdout
    d = json.loads(p)

    # Uncomment this if you want to do diagnostics on the JSON FFmpeg outputs
    #print(json.dumps(d, indent=3, sort_keys=True))
    
    
    # Get the container type from the FFProbe output
    original_container = (d["format"]["format_name"])
    #print (original_container)
    
    # Get the video codec from the first video stream
    for stream in d['streams']:
        if stream['codec_type']=="video":
            original_video_codec = stream['codec_name']
            break
    #print(original_audio_codec)
    
     # Get the video codec from the first audio stream
    for stream in d['streams']:
        if stream['codec_type']=="audio":
            original_audio_codec = stream['codec_name']
            break
    #print(original_audio_codec)

    # Get the subtitle format from the first subtitle stream
    for stream in d['streams']:
        if stream['codec_type']=="subtitle":
            ffmpeg_subtitle_format = stream['codec_name']
            break
    #print(original_audio_codec)
    
    
    
    
    # Here we check the output of ffprobe against the configurations for the library
    # See the template.json
    
    # Part 1 (bellow) checks the container, codecs, formats and builds the relevant ffmpeg string 
    
    # Create an empty string variable that will become our FFmpeg variables
    encode = str()

    # Determine if the container needs to be changed
    if original_container != (filename["ffmpeg_container"]):
        print (ffprobe_path + ' is using ' + original_container + ', not' + (filename["ffmpeg_container"]))

    # Determine if the video needs to be re-encoded
    if original_video_codec != (filename["ffmpeg_video_codec"]):
        print (ffprobe_path + ' is using ' + original_video_codec + ', not' + (filename["ffmpeg_video_codec"]))
        encode = encode + ' ' + (filename["ffmpeg_video_string"])
    
    # Determine if the audio needs to be re-encoded
    if original_audio_codec != (filename["ffmpeg_audio_codec"]):
        print (ffprobe_path + ' is using ' + original_audio_codec + ', not ' + (filename["ffmpeg_audio_codec"]))
        encode = encode + ' ' + (filename["ffmpeg_audio_string"])
        
    # Determine if the subtitles needs to be re-formatted
    if ffmpeg_subtitle_format != (filename["ffmpeg_subtitle_format"]):
        print (ffprobe_path + ' is using ' + ffmpeg_subtitle_format + ', not ' + (filename["ffmpeg_subtitle_format"]))
        encode = encode + ' ' + (filename["ffmpeg_subtitle_string"])  
  
    ffmpeg_json = {'ffmpeg_encoding_string':encode}
    ffmpeg_json.update(filename)
    print(json.dumps(ffmpeg_json, indent=3, sort_keys=True))


    # Part 2 determines if the string is needed
    if original_container == (filename["ffmpeg_container"]) and original_video_codec == (filename["ffmpeg_video_codec"]) and original_audio_codec == (filename["ffmpeg_audio_codec"]) and ffmpeg_subtitle_format == (filename["ffmpeg_subtitle_format"]): 
        print (ffprobe_path + ' is using all of the correct containers and codecs')
    else:
        print (ffprobe_path + ' is going to have to be processed by FFmpeg')        
        return ffmpeg_json
    
      

    
    
    
    


In [14]:
json_file = f'C:\\Users\\Chase\\media\\Newfolder\\test.json'
f = open(json_file)
filename = json.load(f)
#'C:\\Users\\Chase\\media\\Newfolder\\boil_watch\\test.mkv'
ffprober(filename)

Going to execute FFprobe on: C:\Users\Chase\media\Newfolder\boil_watch\asd asd\newtest.mkv
C:\Users\Chase\media\Newfolder\boil_watch\asd asd\newtest.mkv is using hevc, notav1
C:\Users\Chase\media\Newfolder\boil_watch\asd asd\newtest.mkv is using ass, not subrip
{
   "config_name": "default",
   "ffmeg_container_extension": "mkv",
   "ffmpeg_audio_codec": "opus",
   "ffmpeg_audio_string": "-c:a libopus",
   "ffmpeg_container": "matroska,webm",
   "ffmpeg_container_string": "matroska,webm",
   "ffmpeg_encoding_string": " -c:v libsvtav1 -crf 20 -preset 4 -g 240 -pix_fmt yuv420p10le -c:s srt",
   "ffmpeg_settings": "-hide_banner -loglevel 8 -stats",
   "ffmpeg_subtitle_format": "subrip",
   "ffmpeg_subtitle_string": "-c:s srt",
   "ffmpeg_video_codec": "av1",
   "ffmpeg_video_string": "-c:v libsvtav1 -crf 20 -preset 4 -g 240 -pix_fmt yuv420p10le",
   "file_name": "newtest.mkv",
   "file_path": "C:\\Users\\Chase\\media\\Newfolder\\boil_watch\\asd asd",
   "watch_folder": "C:\\Users\\Chase\\

{'ffmpeg_encoding_string': ' -c:v libsvtav1 -crf 20 -preset 4 -g 240 -pix_fmt yuv420p10le -c:s srt',
 'config_name': 'default',
 'ffmpeg_audio_codec': 'opus',
 'ffmpeg_audio_string': '-c:a libopus',
 'ffmpeg_container': 'matroska,webm',
 'ffmeg_container_extension': 'mkv',
 'ffmpeg_container_string': 'matroska,webm',
 'ffmpeg_subtitle_format': 'subrip',
 'ffmpeg_subtitle_string': '-c:s srt',
 'ffmpeg_video_codec': 'av1',
 'ffmpeg_video_string': '-c:v libsvtav1 -crf 20 -preset 4 -g 240 -pix_fmt yuv420p10le',
 'watch_folder': 'C:\\Users\\Chase\\media\\Newfolder',
 'ffmpeg_settings': '-hide_banner -loglevel 8 -stats',
 'file_name': 'newtest.mkv',
 'file_path': 'C:\\Users\\Chase\\media\\Newfolder\\boil_watch\\asd asd'}

NameError: name 'ffmpeger' is not defined

In [119]:
import json
from pathlib import Path
import os

def ffmpeger(ffmpeg_json):
    
    print ('================= ffmpeger JSON inputs =================')
    print(json.dumps(ffmpeg_json, indent=3, sort_keys=True))
    
    # ffmpeg is ffmpeg + settings + input file + encoding settings + output file
    # So we're going to grab each of those pieces

    # Need to get the ffmpeg settings
    ffmpeg_settings = (ffmpeg_json["ffmpeg_settings"])
    #print ('ffmpeg settings are: ' + ffmpeg_settings)
        
    # Need to get the input filepath
    file_path = (ffmpeg_json["file_path"])
    file_name = (ffmpeg_json["file_name"])
    ffmeg_input_file = os.path.join(file_path,file_name)
    #print ('input file is: ' + ffmeg_input_file)
    
    # Need to get the encoding settings     
    ffmpeg_encoding_settings = (ffmpeg_json["ffmpeg_encoding_string"])
    #print ('encoding settings are: ' + ffmpeg_encoding_settings)
    
    # Need to get the output filepath   
    file_name = Path(file_name).stem
    output_extension = (ffmpeg_json["ffmeg_container_extension"])
    #print ('filename is currently: ' + file_name)
    ffmpeg_output_file = '/boil_hold/' + file_name + '.' + output_extension
    #print ('file name with path is: ' + ffmpeg_output_file)
    
    # All together now
    print ('=============== assembled ffmpeg command ===============')
    ffmpeg_command = 'ffmpeg ' + ffmpeg_settings + ' -i "' + ffmeg_input_file + '"' + ffmpeg_encoding_settings + ' ".' + ffmpeg_output_file + '"'
    print (ffmpeg_command)    
    print ('=============== executing ffmpeg =======================')

    #print (ffmpeg_command)
    print ('Please hold')
    os.system(ffmpeg_command)
    
    # Would love to revisit this as a subprocess, but was having issues getting everything to slice as desired
    #print ('=============== assembled ffmpeg command ===============')
    #ffmpeg_settings = ffmpeg_settings.split()
    #ffmpeg_settings = ', '.join(ffmpeg_settings)
    #ffmpeg_encoding_settings = ffmpeg_encoding_settings.split()
    #ffmeg_input_file = '"' + ffmeg_input_file + '"'
    #ffmpeg_output_file = '"' + ffmpeg_output_file + '"'
    #ffmpeg_command = 'ffmpeg', ffmpeg_settings, '-i', ffmeg_input_file, ffmpeg_encoding_settings,ffmpeg_output_file
    #print (ffmpeg_command)    
    #print ('=============== executing ffmpeg =======================')
    
    #p = subprocess.run(cmnd, capture_output=True).stdout
    #d = json.loads(p)

    print ('=============== Checking output =======================')
    
    if os.path.exists(ffmeg_input_file and ffmpeg_output_file):
        print( ffmeg_input_file + ' and ' + ffmpeg_output_file + ' Files Exists')
        input_file_stats = os.stat(ffmeg_input_file)
        print(f'Original file Size in MegaBytes is {input_file_stats.st_size / (1024 * 1024)}') 
        output_file_stats = os.stat(ffmpeg_output_file)
        print(f'Encoded file Size in MegaBytes is {output_file_stats.st_size / (1024 * 1024)}') 
        print('Removing ' + ffmeg_input_file)
        #os.remove(ffmeg_input_file) 
        print('Moving ' + ffmpeg_output_file + ' to ' + ffmeg_input_file)
        #os.rename(ffmpeg_output_file, ffmeg_input_file)
        print ('Done')    
    else:
         print("Something Broke")
    
    
    

In [120]:
json_file = f'C:\\Users\\Chase\\media\\Newfolder\\ffmpeg.json'
f = open(json_file)
ffmpeg_json = json.load(f)
#'C:\\Users\\Chase\\media\\Newfolder\\boil_watch\\test.mkv'
ffmpeger(ffmpeg_json)

================= ffmpeger JSON inputs =================
{
   "config_name": "default",
   "ffmeg_container_extension": "mkv",
   "ffmpeg_audio_codec": "opus",
   "ffmpeg_audio_string": "-c:a libopus",
   "ffmpeg_container": "matroska,webm",
   "ffmpeg_container_string": "matroska,webm",
   "ffmpeg_encoding_string": " -c:v libsvtav1 -crf 20 -preset 4 -g 240 -pix_fmt yuv420p10le -c:s srt",
   "ffmpeg_settings": "-hide_banner -loglevel 8 -stats",
   "ffmpeg_subtitle_format": "subrip",
   "ffmpeg_subtitle_string": "-c:s srt",
   "ffmpeg_video_codec": "av1",
   "ffmpeg_video_string": "-c:v libsvtav1 -crf 20 -preset 4 -g 240 -pix_fmt yuv420p10le",
   "file_name": "newtest.mkv",
   "file_path": "C:\\Users\\Chase\\media\\Newfolder\\boil_watch\\asd asd",
   "watch_folder": "C:\\Users\\Chase\\media\\Newfolder"
}
=============== assembled ffmpeg command ===============
ffmpeg -hide_banner -loglevel 8 -stats -i "C:\Users\Chase\media\Newfolder\boil_watch\asd asd\newtest.mkv" -c:v libsvtav1 -crf 20

In [129]:
import json
from pathlib import Path
import os

ffmeg_input_file = 'C:\\Users\\Chase\\media\\Newfolder\\boil_watch\\asd asd\\newtest.mkv'
ffmpeg_output_file = './/boil_hold//newtest.mkv'



C:\Users\Chase\media\Newfolder\boil_watch\asd asd\newtest.mkv and .//boil_hold//newtest.mkv Files Exists
Original file Size in MegaBytes is 293.1825485229492
Encoded file Size in MegaBytes is 256.9948396682739
Removing C:\Users\Chase\media\Newfolder\boil_watch\asd asd\newtest.mkv
Moving .//boil_hold//newtest.mkv to C:\Users\Chase\media\Newfolder\boil_watch\asd asd\newtest.mkv
Done
